## HOMEWORK 1 ##
Custom CNN



In [ ]:
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive/My Drive/[2023-2024] AN2DL/Homework 1
%cd /gdrive/My Drive

In [2]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [3]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.14.0


In [4]:
# Import other libraries
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns

## Load Data + Display images ##

In [ ]:
# Conditional check for unzipping
unzip = False

# Unzip the 'animals.zip' file if the 'unzip' flag is True
if unzip:
    !unzip public_data.zip

In [5]:
# Reading the .npz file
data = np.load('public_data.npz', allow_pickle=True)

In [ ]:
# Getting the images and labels
data_array = data['data']
labels_array = data['labels']

# Check the shapes of the array
print("Data array shape: ", data_array.shape)
print("Data array shape: ", labels_array.shape)

In [7]:
# Getting images in a np.array
images = []

for img in data_array:
  img=(img).astype(np.float32)
  images.append(img)

In [ ]:
# Number of images to display
num_img = 10

# Create subplots for displaying leaves
fig, axes = plt.subplots(2, num_img//2, figsize=(8, 4))
for i in range(num_img):
    ax = axes[i%2, i%num_img//2]
    ax.imshow(np.clip(images[i]/255, 0, 255))  # Display clipped item images
    ax.axis('off')
plt.tight_layout()
plt.show()

## Creating labels and sets ##

In [9]:
y = []
count_h = 0
# Create an array with zeros and ones based on the tag
for i in range(len(images)):
    if labels_array[i] == 'healthy':
        count_h = count_h + 1
        y.append(0)
    else:
        y.append(1)

# Convert labesl to one-hot encoding format
y = tfk.utils.to_categorical(y,2)

In [ ]:
# Count the number of healthy labels
print("Total number of labels: " + str(len(images)))
print("Number of healthy labels: " + str(count_h))
print("Number of unhealthy labels: " + str(len(images) - count_h))

In [ ]:
# Creating the sets
X = np.array(images)

# Split data into train, validation and test set
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=0.1, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=len(X_test), stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

## Adding noise ONLY to the train set

In [12]:
# Adding noise
X_train_noise = []

for image in X_train:
    # Adding noise to the image
    VARIABILITY = 35
    deviation = VARIABILITY * random.random()
    noise = np.random.normal(0, deviation, image.shape)
    image += noise
    np.clip(image, 0., 255., out=image)
    X_train_noise.append(image)

X_train = np.array(X_train_noise)

In [ ]:
# Display leaves with noise
# Number of images to display
num_img = 10

# Create subplots for displaying leaves
fig, axes = plt.subplots(2, num_img//2, figsize=(8, 4))
for i in range(num_img):
    ax = axes[i%2, i%num_img//2]
    ax.imshow(np.clip(X_train[i]/255, 0, 255))  # Display clipped item images
    ax.axis('off')
plt.tight_layout()
plt.show()


## Creating the model ##

In [ ]:
# Defining input and output shapes
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]

# Print the shapes of the resulting datasets
print(input_shape)
print(output_shape)

In [15]:
def build_model(input_shape=input_shape, output_shape=output_shape):
    tf.random.set_seed(seed)

    # Data augmentation
    data_aug = tf.keras.Sequential([
        tfkl.RandomFlip('horizontal_and_vertical'),
        tfkl.RandomRotation(0.5),
        tfkl.RandomTranslation(0.2, 0.2),
        tfkl.RandomZoom(0.2),
        tfkl.RandomBrightness(0.2, value_range=(0,255)),
        tfkl.RandomContrast(0.2),
    ], name='data_aug')

    # Build the input layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    x = data_aug(input_layer)

    # 96x96x3
    x = tfkl.Conv2D(filters=55, kernel_size=3, activation='relu', padding='valid', name='conv01')(x)
    x = tfkl.Conv2D(filters=70, kernel_size=3, activation='relu', padding='valid', name='conv02')(x)
    x = tfkl.Conv2D(filters=85, kernel_size=3, activation='relu', padding='valid', name='conv03')(x)
    x = tfkl.Conv2D(filters=100, kernel_size=3, activation='relu', padding='valid', name='conv04')(x)
    x = tfkl.MaxPooling2D(pool_size =(2,2), name='mp0')(x)
    # 44x44x80

    x = tfkl.Normalization()(x)

    # 44x44x80
    x = tfkl.Conv2D(filters=155, kernel_size=3, activation='relu', padding='valid', name='conv11')(x)
    x = tfkl.Conv2D(filters=170, kernel_size=3, activation='relu', padding='valid', name='conv12')(x)
    x = tfkl.Conv2D(filters=185, kernel_size=3, activation='relu', padding='valid', name='conv13')(x)
    x = tfkl.Conv2D(filters=200, kernel_size=3, activation='relu', padding='valid', name='conv14')(x)
    x = tfkl.MaxPooling2D(pool_size =(2,2), name='mp1')(x)
    # 18x18x200

    x = tfkl.Normalization()(x)

    # 18x18x200
    conv1_1x1 = tfkl.Conv2D(filters=150, kernel_size=1, activation='relu', padding='same', name='conv1_1x1')(x)
    conv2_1x1 = tfkl.Conv2D(filters=50, kernel_size=1, activation='relu', padding='same', name='conv2_1x1')(x)
    conv2_3x3 = tfkl.Conv2D(filters=150, kernel_size=1, activation='relu', padding='same', name='conv2_3x3')(conv2_1x1)
    conv3_1x1 = tfkl.Conv2D(filters=50, kernel_size=1, activation='relu', padding='same', name='conv3_1x1')(x)
    conv3_5x5 = tfkl.Conv2D(filters=150, kernel_size=1, activation='relu', padding='same', name='conv3_5x5')(conv3_1x1)
    conv4_1x1 = tfkl.Conv2D(filters=50, kernel_size=1, activation='relu', padding='same', name='conv4_1x1')(x)
    conv4_7x7 = tfkl.Conv2D(filters=150, kernel_size=1, activation='relu', padding='same', name='conv4_7x7')(conv4_1x1)
    mp5_3x3 = tfkl.MaxPooling2D(pool_size =(3,3), strides=1, padding='same', name='mp5_3x3')(x)
    conv5_1x1 = tfkl.Conv2D(filters=150, kernel_size=1, activation='relu', padding='same', name='conv5_1x1')(mp5_3x3)

    # Concatenation of the previous layers
    x = tfkl.Concatenate(name='Concatenation')([conv1_1x1, conv2_3x3, conv3_5x5, conv4_7x7, conv5_1x1])
    # 18x18x750

    # GAP
    x = tfkl.GlobalAveragePooling2D(name='gap')(x)

    # Leaky ReLU after the gap
    x = tfkl.LeakyReLU()(x)

    # Dense layer + dropout & weight decay series
    droput_rate = 1/6;
    l2_lambda = 2e-6;

    x = tfkl.Dense(units=600, activation='gelu', kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda),name='dense1')(x)
    x = tfkl.Dropout(droput_rate, seed=seed)(x)

    x = tfkl.Dense(units=325, activation='gelu', kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda),name='dense2')(x)
    x = tfkl.Dropout(droput_rate, seed=seed)(x)

    x = tfkl.Dense(units=50, activation='gelu', kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda),name='dense3')(x)
    x = tfkl.Dropout(droput_rate, seed=seed)(x)

    # Output layer
    output_layer = tfkl.Dense(units=2, activation='softmax',name='Output')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='BuddysCNNV final')

    # Compile the model
    learning_rate = 1e-3
    model.compile(loss=tfk.losses.CategoricalCrossentropy(label_smoothing=0.15), optimizer=tfk.optimizers.Adam(learning_rate), metrics=['accuracy'])

    # Return the model
    return model

In [ ]:
# Print important the summary of the model and it's graph
model = build_model()
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
# Class weights to balance classes
ymax = np.argmax(y_train, axis=1)
weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(np.argmax(y_train, axis=1)),
                                                 y=ymax)
print(weights)

class_weights = {0: weights[0], 1: weights[1]}

In [ ]:
# Training
history = model.fit(
    x = X_train,
    y = y_train,
    class_weight=class_weights,
    batch_size = 256,
    epochs = 750,
    validation_data = (X_val, y_val),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=60, restore_best_weights=True),
                 tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=5, factor=0.94, min_lr=1e-6)]
).history

In [ ]:
# Printing the best validation accuracy
max(history['val_accuracy'])

# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(history['val_accuracy'])

# Plot the training history
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Re-trained', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], label='Re-trained', alpha=.8, color='#ff7f0e')
plt.plot(best_epoch, history['val_accuracy'][best_epoch], marker='*', alpha=0.8, markersize=10, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Save the trained model
model.save('BuddysCNN')

In [ ]:
# Delete the model
del model

In [ ]:
# Load the saved LeNet model
model = tfk.models.load_model('BuddysCNN')

In [ ]:
# Predict labels for the test set
predictions = model.predict(X_test, verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

# Compute the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))

# Compute classification metrics
accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')

# Display the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm.T, xticklabels=list(['healthy', 'unhealthy']), yticklabels=list(['healthy', 'unhealthy']), cmap='Blues', annot=True)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()